In [1]:
import pandas as pd
import numpy as np
import sys
sys.path.insert(0, "../")
import alphalens


from factor_cal.config_loader import basic_config as cfg
from factor_cal.table.ddb_table import PriceTable, SecLevelFacTable
from factor_cal.utils import ddb_utils as du


In [2]:
# read config file
config = cfg.BasicConfig('config/config.yml')
# obtain the ddb session
s = du.DDBSessionSingleton().session

factor_names = []
for facType in config['factors']:
    factor_names += list(config['factors'][facType].keys())

In [5]:
factor_names = list(set(factor_names))
len(factor_names)

61

In [9]:
excluded_factors = ['close_ret']
factor_names = [x for x in factor_names if x not in excluded_factors]

# For test

factor_names = ['close_ret', 'pv_corr']

In [46]:
start_date = '2023.09.22'
end_date = '2024.02.20'
start_time = '09:45:00'
end_time = '14:45:00'
dates = pd.date_range(start_date, end_date)
date = dates[0].strftime('%Y.%m.%d')
factor_name = factor_names[0]
fac_tb = SecLevelFacTable(config['factor_dbPath'], config['factor_tbName'])
fac = fac_tb.load_factor(factor_name, date, start_time, end_time, sec_list=None)
fac_df = s.loadTable(tableName=fac).toDF()

In [47]:
fac_df.rename(columns={'value': factor_name}, inplace=True)
fac_df.drop('factorname', axis=1, inplace=True)

In [21]:
factor_name = factor_names[1]
fac_tb = SecLevelFacTable(config['factor_dbPath'], config['factor_tbName'])
fac = fac_tb.load_factor(factor_name, date, config['start_time'], config['end_time'], sec_list=None)
fac_df1 = s.loadTable(tableName=fac).toDF()

In [22]:
fac_df1.rename(columns={'value': factor_name}, inplace=True)
fac_df1.drop('factorname', axis=1, inplace=True)
fac_df1.head(5)

,tradetime,securityid,pv_corr
0,2023-09-22 09:30:00,000001.SZ,NaN
1,2023-09-22 09:30:03,000001.SZ,NaN
2,2023-09-22 09:30:06,000001.SZ,NaN
3,2023-09-22 09:30:09,000001.SZ,NaN
4,2023-09-22 09:30:12,000001.SZ,NaN


In [23]:
res = fac_df.merge(fac_df1, how='outer', on=['tradetime', 'securityid'])

In [24]:
res.head(5)

,tradetime,securityid,close_ret,pv_corr
0,2023-09-22 09:30:00,000001.SZ,NaN,NaN
1,2023-09-22 09:30:03,000001.SZ,NaN,NaN
2,2023-09-22 09:30:06,000001.SZ,0.000906,NaN
3,2023-09-22 09:30:09,000001.SZ,0.001810,NaN
4,2023-09-22 09:30:12,000001.SZ,-0.000903,NaN


In [48]:
price_info = config['price_info']
pc_tb = PriceTable(price_info['price_dbPath'], price_info['price_tbName'], price_info['time_col'], price_info['sec_col'], price_info['price_cols'])
price = pc_tb.load_price(date, config['start_time'], config['end_time'], sec_list=None)
price_df = s.loadTable(tableName=price).toDF()
price_df = price_df.set_index(['tradetime', 'securityid'])
prices = price_df['close'].unstack()

In [49]:
fac_df.set_index(['tradetime', 'securityid'], inplace=True)
fac_df = fac_df.sort_index(level=0)

In [50]:
ret_df = alphalens.utils.compute_forward_returns(fac_df, prices, periods=[20, 60, 100], filter_zscore=20, cumulative_returns=True)

In [40]:
ret_df.index.set_names(['tradetime', 'securityid'], inplace=True)
ret_df.reset_index(inplace=True)

In [52]:
ret_df[~(ret_df.isna().sum(axis=1) == 3)]

1m        3m        5m
date                asset                                  
2023-09-22 09:45:00 000001.SZ -0.000904 -0.001808  0.000000
                    000002.SZ -0.000755  0.000000  0.000000
                    000004.SZ  0.005611  0.000000 -0.002494
                    000005.SZ  0.006803  0.000000  0.006803
                    000006.SZ -0.002174 -0.002174  0.002174
...                                 ...       ...       ...
2023-09-22 14:45:00 688799.SH -0.000492 -0.009109 -0.008616
                    688800.SH -0.002284 -0.000507 -0.001269
                    688819.SH  0.000287  0.001721  0.001721
                    688981.SH -0.000195  0.000000  0.000586
                    689009.SH -0.001425 -0.000570 -0.000285

[21257918 rows x 3 columns]

In [54]:
import pandas as pd
date = '2023.09.22'
df = pd.read_pickle(f'/home/wangzirui/workspace/data/fac_ret_{date}.pkl')

In [55]:
df.head(5)

,tradetime,securityid,1m,3m,5m,pv_corr,td_pv_corr_5min
0,2023-09-22 09:45:00,000001.SZ,-0.000904,-0.001808,0.000000,0.000000,-0.190327
1,2023-09-22 09:45:00,000002.SZ,-0.000755,0.000000,0.000000,0.278530,-0.087164
2,2023-09-22 09:45:00,000004.SZ,0.005611,0.000000,-0.002494,0.284619,-0.196392
3,2023-09-22 09:45:00,000005.SZ,0.006803,0.000000,0.006803,-0.152224,-0.001619
4,2023-09-22 09:45:00,000006.SZ,-0.002174,-0.002174,0.002174,NaN,0.010128
